<a href="https://colab.research.google.com/github/ahsaniqbal/CharacterLevelLM/blob/master/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'MachineLearning/CharLM/'

Mounted at /content/gdrive


In [0]:
import os
if not os.path.exists(base_dir):
  os.makedirs(base_dir)

In [4]:
!rm -r CharacterLevelLM
!git clone https://github.com/ahsaniqbal/CharacterLevelLM
import torch

Cloning into 'CharacterLevelLM'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 63 (delta 23), reused 50 (delta 16), pack-reused 0
Unpacking objects: 100% (63/63), done.


In [0]:
from CharacterLevelLM.Datasets.CharDataset import CharDataset, CustomCollate
from CharacterLevelLM.Models.LSTMBased import LSTMModel

In [0]:
dataset_train = CharDataset('./CharacterLevelLM/data.txt', 100, train_size=0.95)
dataset_test = CharDataset('./CharacterLevelLM/data.txt', 100, is_train=False, train_size=0.95)

In [7]:
dataset_test.get_vocab_size()

62

In [0]:
model = LSTMModel(dataset_train.get_vocab_size(), 128, dataset_test.get_vocab_size())

In [0]:
from torch.utils.data import DataLoader

loader_train = DataLoader(dataset_train, batch_size=50, shuffle=True, collate_fn=CustomCollate())
loader_test = DataLoader(dataset_test, batch_size=50, shuffle=True, collate_fn=CustomCollate())

In [0]:
from torch.optim import Adam
optimizer = Adam(model.parameters())

In [0]:
from torch import nn
criterion = nn.CrossEntropyLoss(reduction = 'none')

In [0]:
import numpy as np
best_loss = np.inf

In [17]:
model = model.cuda()
for i in range(50):
  model.train()
  for data in loader_train:
    optimizer.zero_grad()
    X, Y, mask = data
    X, Y, mask = X.cuda(), Y.cuda(), mask.cuda()
    
    Y = Y.view(Y.shape[0] * Y.shape[1])
    mask = mask.view(mask.shape[0] * mask.shape[1])

    loss = (criterion(model(X), Y) * mask).mean()
    loss.backward()
    optimizer.step()
  model.eval()

  test_loss = 0 
  with torch.no_grad():
    for idx, data in enumerate(loader_test):
      X, Y, mask = data
      X, Y, mask = X.cuda(), Y.cuda(), mask.cuda()

      Y = Y.view(Y.shape[0] * Y.shape[1])
      mask = mask.view(mask.shape[0] * mask.shape[1])

      test_loss *= idx
      test_loss += (criterion(model(X), Y) * mask).mean().item()
      test_loss /= (idx + 1)
  print(test_loss)
  if test_loss < best_loss:
    best_loss = test_loss
    torch.save({'state_dict': model.state_dict()}, base_dir + '/model.pth')


1.8043779134750366
1.8018550872802734
1.8042256832122803
1.8023762702941895
1.8022997379302979
1.8033647537231445
1.7983475923538208
1.807036280632019
1.8050304651260376
1.8080040216445923
1.8046661615371704
1.803824543952942
1.8044979572296143
1.8033077716827393
1.8052600622177124
1.8099831342697144
1.8024168014526367
1.8137800693511963
1.804850697517395
1.8075615167617798
1.8101682662963867
1.807334303855896
1.8141589164733887
1.8126269578933716
1.815405011177063
1.8095300197601318
1.811915636062622
1.814003825187683
1.8111811876296997
1.807555079460144
1.815881609916687
1.816457748413086
1.8121964931488037
1.8194541931152344
1.82039213180542
1.8167394399642944
1.8190885782241821
1.8128448724746704
1.8262298107147217
1.8197487592697144
1.814258098602295
1.821378469467163
1.8186347484588623
1.819235920906067
1.8267265558242798
1.8306869268417358
1.8294165134429932
1.8248339891433716
1.8255943059921265
1.8325155973434448
